In [29]:
import numpy as np
np.random.seed(123)

import pandas as pd
import yfinance as yahooFinance
import datetime

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K


class Model:
    def __init__(self):
        self.data = None
        self.model = None
        
    def __build_model(self, input_shape, outputs):
        '''
        Builds and returns the Deep Neural Network that will compute the allocation ratios
        that optimize the Sharpe Ratio of the portfolio
        
        inputs: input_shape - tuple of the input shape, outputs - the number of assets
        returns: a Deep Neural Network model
        '''
        model = Sequential([
            LSTM(64, input_shape=input_shape),
            Flatten(),
            Dense(outputs, activation='softmax')
        ])

        def sharpe_loss(_, y_pred):
            # make all time-series start at 1
            data = tf.divide(self.data, self.data[0])  
            
            # value of the portfolio after allocations applied
            portfolio_values = tf.reduce_sum(tf.multiply(data, y_pred), axis=1) 
            
            portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

            sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
            # since we want to maximize Sharpe, while gradient descent minimizes the loss, 
            #   we can negate Sharpe (the min of a negated function is its max)
            return -sharpe
        
        model.compile(loss=sharpe_loss, optimizer='adam')
        return model
    
    def get_allocations(self, data: pd.DataFrame, epochs):
        '''
        Computes and returns the allocation ratios that optimize the Sharpe over the given data
        
        input: data - DataFrame of historical closing prices of various assets
        
        return: the allocations ratios for each of the given assets
        '''
        # data with returns
        data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
        data = data.iloc[1:]
        self.data = tf.cast(tf.constant(data), float)
        
        if self.model is None:
            self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
        fit_predict_data = data_w_ret[np.newaxis,:]        
        self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=epochs, shuffle=False, verbose=False)
        return self.model.predict(fit_predict_data)[0]

In [23]:
labels = ["vti", "agg","dbc","vix"]

In [44]:
def get_data(startDate, endDate):
    vti = yahooFinance.Ticker("VTI").history(start=startDate,end=endDate).reset_index()
    agg = yahooFinance.Ticker("AGG").history(start=startDate,end=endDate).reset_index()
    dbc = yahooFinance.Ticker("DBC").history(start=startDate,end=endDate).reset_index()
    vix = yahooFinance.Ticker("^VIX").history(start=startDate,end=endDate).reset_index()
    all_assets = [vti, agg, dbc, vix]
    data = pd.DataFrame()
    for i, asset in enumerate(all_assets):
        asset = asset.reset_index()
        lb = labels[i]
        data[lb] = asset['Close']
    return data

In [62]:
money = 1
startDate = datetime.datetime(2006, 2, 6) 
endDate = datetime.datetime(2011, 12, 31)
model = Model()

for i in range(100):
    startDate += datetime.timedelta(days=1)
    endDate += datetime.timedelta(days = 1)
    data = get_data(startDate, endDate)
    weights = model.get_allocations(data.iloc[:-1], epochs = 10)
    returns = data.iloc[-1] / data.iloc[-2]
    port_returns = weights@returns
    if port_returns != port_returns:
        continue
    money*=port_returns
    print(i, money)

1/1 [==============================] - 0s 120ms/step
0 1.0236654178329128
1/1 [==============================] - 0s 121ms/step
1 1.0551364386883524
1/1 [==============================] - 0s 22ms/step
2 1.08965893416407
1/1 [==============================] - 0s 21ms/step
3 1.0698088216408894
1/1 [==============================] - 0s 22ms/step
4 1.034977361026227
1/1 [==============================] - 0s 22ms/step
5 1.000564963081671
1/1 [==============================] - 0s 21ms/step
6 0.9610316717293886
1/1 [==============================] - 0s 22ms/step
7 0.9230545889893863
1/1 [==============================] - 0s 22ms/step
8 0.8865738264527135
1/1 [==============================] - 0s 22ms/step
9 0.9054624794843195
1/1 [==============================] - 0s 22ms/step
10 0.8891548403113718
1/1 [==============================] - 0s 23ms/step
11 0.9046112588040753
1/1 [==============================] - 0s 22ms/step
12 0.8797047495512809
1/1 [==============================] - 0s 22ms/ste

KeyboardInterrupt: 

### Notes:
- Prone to overfitting. Epochs = 100 loses money quickly. Epochs = 5 makes money steadily. 